In [1]:
import os
import joblib
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [29]:
data_path = os.path.join('datasets', 'nyc education + jobs')

def load_data(filename, path):
    csv_path = os.path.join(path, filename)
    return pd.read_csv(csv_path)

census_2010 = load_data("nyc_census_2010.csv", data_path)
census_2018 = load_data("nyc_census_2018.csv", data_path)

In [30]:
census_2010 = census_2010.drop(columns=['year', 'NAME', 'state', 'county', 'tract', 'B08528_010E'])
census_2018 = census_2018.drop(columns=['year', 'NAME', 'state', 'county', 'tract', 'B08528_010E'])

13.5%
2018 price * 0.865 = 2010 price

In [31]:
def column_names(dataframe, year):
    year_columns = []
    columns = ['population', 'household_income', 'home_value', 
                        'no_nonh_caucasians', 'no_nonh_blacks/aas', 'no_amerinds_alskns', 'no_nonh_asians',
                        'no_nonh_hawaii_pacific', 'no_nonh_others', 'no_nonh_multi', 'no_hisp_latin',
                        'construction', 'retail', 'information', 'finance', 'edu_health_social',
                        'arts_recreation_accom', 'public_admin', 'armed_forces', 'bachelors', 'male<25_bachelors',
                        'female<25_bachelors']
    year_columns.append('geoid')
    for i in columns:
        year_columns.append(year + '_' + i)
    dataframe.columns = year_columns

column_names(census_2010, '2010')
column_names(census_2018, '2018')

In [34]:
census_2010.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701 entries, 0 to 4700
Data columns (total 23 columns):
geoid                          4701 non-null int64
2010_population                4701 non-null float64
2010_household_income          4701 non-null float64
2010_home_value                4701 non-null float64
2010_no_nonh_caucasians        4701 non-null float64
2010_no_nonh_blacks/aas        4701 non-null float64
2010_no_amerinds_alskns        4701 non-null float64
2010_no_nonh_asians            4701 non-null float64
2010_no_nonh_hawaii_pacific    4701 non-null float64
2010_no_nonh_others            4701 non-null float64
2010_no_nonh_multi             4701 non-null float64
2010_no_hisp_latin             4701 non-null float64
2010_construction              4701 non-null float64
2010_retail                    4701 non-null float64
2010_information               4701 non-null float64
2010_finance                   4701 non-null float64
2010_edu_health_social         4701 non-null fl

In [35]:
census_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 23 columns):
geoid                          4700 non-null int64
2018_population                4700 non-null float64
2018_household_income          4700 non-null float64
2018_home_value                4700 non-null float64
2018_no_nonh_caucasians        4700 non-null float64
2018_no_nonh_blacks/aas        4700 non-null float64
2018_no_amerinds_alskns        4700 non-null float64
2018_no_nonh_asians            4700 non-null float64
2018_no_nonh_hawaii_pacific    4700 non-null float64
2018_no_nonh_others            4700 non-null float64
2018_no_nonh_multi             4700 non-null float64
2018_no_hisp_latin             4700 non-null float64
2018_construction              4700 non-null float64
2018_retail                    4700 non-null float64
2018_information               4700 non-null float64
2018_finance                   4700 non-null float64
2018_edu_health_social         4700 non-null fl

In [36]:
census_data = pd.merge(census_2010, census_2018, on='geoid')

In [37]:
census_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4700 entries, 0 to 4699
Data columns (total 45 columns):
geoid                          4700 non-null int64
2010_population                4700 non-null float64
2010_household_income          4700 non-null float64
2010_home_value                4700 non-null float64
2010_no_nonh_caucasians        4700 non-null float64
2010_no_nonh_blacks/aas        4700 non-null float64
2010_no_amerinds_alskns        4700 non-null float64
2010_no_nonh_asians            4700 non-null float64
2010_no_nonh_hawaii_pacific    4700 non-null float64
2010_no_nonh_others            4700 non-null float64
2010_no_nonh_multi             4700 non-null float64
2010_no_hisp_latin             4700 non-null float64
2010_construction              4700 non-null float64
2010_retail                    4700 non-null float64
2010_information               4700 non-null float64
2010_finance                   4700 non-null float64
2010_edu_health_social         4700 non-null fl

In [38]:
census_data['2018_home_value']

0        710600.0
1       2000001.0
2        466600.0
3        631000.0
4        332800.0
          ...    
4695     161100.0
4696     158100.0
4697     125600.0
4698     141600.0
4699     161200.0
Name: 2018_home_value, Length: 4700, dtype: float64

In [39]:
census_data['2018_home_value'] = census_data['2018_home_value'] * 0.865

In [40]:
census_data['2018_home_value']

0        614669.000
1       1730000.865
2        403609.000
3        545815.000
4        287872.000
           ...     
4695     139351.500
4696     136756.500
4697     108644.000
4698     122484.000
4699     139438.000
Name: 2018_home_value, Length: 4700, dtype: float64

In [41]:
census_data['percent_change_home_value'] = 100*(census_data['2018_home_value'] - census_data['2010_home_value']) / census_data['2018_home_value']

In [42]:
census_data['percent_change_home_value']

0      -20.617763
1       42.196503
2      -31.389538
3      -32.059397
4      -27.591430
          ...    
4695   -24.648820
4696   -56.921243
4697   -58.683406
4698   -47.774403
4699   -41.281430
Name: percent_change_home_value, Length: 4700, dtype: float64

In [43]:
census_data['2018_>25bachelors'] = census_data['2018_bachelors'] - (census_data['2018_male<25_bachelors'] + census_data['2018_female<25_bachelors'])
census_data['2010_>25bachelors'] = census_data['2010_bachelors'] - (census_data['2010_male<25_bachelors'] + census_data['2010_female<25_bachelors'])

In [45]:
census_data['2018_percent_>25bachelors'] = census_data['2018_>25bachelors'] / census_data['2018_population']
census_data['2010_percent_>25bachelors'] = census_data['2010_>25bachelors'] / census_data['2010_population']

In [46]:
census_data['2018_percent_>25bachelors']

0       0.252292
1       0.313285
2       0.251109
3       0.224265
4       0.199845
          ...   
4695    0.070352
4696    0.086196
4697    0.095834
4698    0.108200
4699    0.096890
Name: 2018_percent_>25bachelors, Length: 4700, dtype: float64

In [47]:
census_data['change_percent_>25bachelors'] = census_data['2018_percent_>25bachelors'] - census_data['2010_percent_>25bachelors']

In [48]:
census_data['change_percent_>25bachelors']

0       0.030841
1       0.023565
2       0.135513
3      -0.018079
4       0.061081
          ...   
4695   -0.042749
4696    0.045309
4697    0.017939
4698    0.002024
4699    0.046221
Name: change_percent_>25bachelors, Length: 4700, dtype: float64

In [53]:
education_97_percentile = census_data['change_percent_>25bachelors'].quantile(0.97)
value_97_percentile = census_data['percent_change_home_value'].quantile(0.97)

In [54]:
education_97_percentile

0.09512611353681051

In [55]:
value_97_percentile

100.01032797688893

In [56]:
census_data['gentrify_elig'] = 0

income_40_percentile = census_data['2010_household_income'].quantile(0.4)
value_40_percentile = census_data['2010_home_value'].quantile(0.4)

census_data.loc[(census_data['2010_household_income'] <= income_40_percentile) & 
               (census_data['2010_home_value'] <= value_40_percentile) &
               (census_data['2010_population'] >= 500), 'gentrify_elig'] = 1

In [57]:
census_data['test_two'] = 0

census_data.loc[(census_data['change_percent_>25bachelors'] >= education_97_percentile) & 
               (census_data['percent_change_home_value'] >= value_97_percentile), 'test_two'] = 1

In [58]:
census_data['gentrified'] = 0

census_data.loc[(census_data['gentrify_elig'] == 1) & 
               (census_data['test_two'] == 1), 'gentrified'] = 1

In [59]:
census_data['gentrified']

0       0
1       0
2       0
3       0
4       0
       ..
4695    0
4696    0
4697    0
4698    0
4699    0
Name: gentrified, Length: 4700, dtype: int64

In [61]:
census_data['gentrified'].value_counts()

0    4699
1       1
Name: gentrified, dtype: int64

In [62]:
census_data.loc[census_data['gentrified'] == 1]

geoid  2010_population  2010_household_income  2010_home_value  \
2913  36061017402           2568.0                25938.0     -666666666.0   

      2010_no_nonh_caucasians  2010_no_nonh_blacks/aas  \
2913                    293.0                   1304.0   

      2010_no_amerinds_alskns  2010_no_nonh_asians  \
2913                      0.0                186.0   

      2010_no_nonh_hawaii_pacific  2010_no_nonh_others  ...  \
2913                          0.0                  0.0  ...   

      2018_female<25_bachelors  percent_change_home_value  2018_>25bachelors  \
2913                      33.0              144077.752414              353.0   

      2010_>25bachelors  2018_percent_>25bachelors  2010_percent_>25bachelors  \
2913               78.0                   0.146352                   0.030374   

      change_percent_>25bachelors  gentrify_elig  test_two  gentrified  
2913                     0.115978              1         1           1  

[1 rows x 54 columns]